### Modelos Sequenciais

Os Modelos Sequenciais (mais conhecidos como Sequence-to-sequence ou pela sigla seq2seq) é uma metodologia baseada em redes neurais onde utiliza-se deste tipo de modelo para obter uma sequência de entrada em um determinado domínio e converta ela para uma representação em outro domínio específico. A partir das sequências de saída basicamente o modelo monta uma distribuição de probabilidade do que possa ser a sequência de saída através da sequência anterior como entrada.

Exemplos clássicos de modelos seq2seq são os tradutores de idiomas, autocomplete (quando ao digitar em um buscador, é sugerir o que a pessoa está querendo escrever) e modelos de autocorrect (dado que a pessoa possa ter digitado algo errado, o modelo sugerir a grafia correta).


### Entendendo melhor

* Duas redes (encoder e decoder)
* Cada rede é responsável por uma tarefa
* O aprendizado do decoder depende do que o encoder aprendeu

![Title](imgs/seq2seq.png)



### Modelos de atenção

* Embora eficientes, os modelos seq2seq baseados apenas em encoder/decoder possuem uma camada intermediária S baseada em um único vetor
* Muitas informações linguísticas armazenadas apenas neste vetor
* Algoritmo tende a degradar quando tem longas sequências
* Os modelos seq2seq baseados em atenção tratam a saída do encoder para filtrar as informações mais importantes a serem aprendidas
* Possui uma rede neural intermediária para aprender estes pontos de foco
![Title](imgs/attention.png)



In [5]:
# Bibliotecas Auxiliares
import string
import numpy as np
from unidecode import unidecode
import pandas as pd
# Bibliotecas de Deep Learning
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Input, TimeDistributed, Dense, Activation, RepeatVector, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [6]:
df = pd.read_csv('../datasets/seq2seq.txt', sep='\t', header=None,
                   names=['english', 'portuguese', 'info'])
df.head()

,english,portuguese,info
0,Go.,Vai.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,Vá.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Hi.,Oi.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
3,Run!,Corre!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
4,Run!,Corra!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...


In [7]:
df = df.loc[1000: 20000]

In [8]:
# função para limpar os textos
def clean_sentence(sentence):
    # Deixa todas as palavras minúsculas
    sentence = sentence.lower()

    # Remove acentuações
    sentence = unidecode(sentence)

    # Remove pontuações
    clean_sentence = sentence.translate(str.maketrans('', '', string.punctuation))

    # Retorna as sentenças limpas
    return clean_sentence

# função para tokenização
def tokenize(sentences):
    # Instância o Tokenizer
    text_tokenizer = Tokenizer()

    # Treino com os textos
    text_tokenizer.fit_on_texts(sentences)

    # Retorna os textos tokenizados
    return text_tokenizer.texts_to_sequences(sentences), text_tokenizer


In [9]:
df["port_clean"] = df['portuguese'].apply(clean_sentence)
df["eng_clean"] = df['english'].apply(clean_sentence)

In [10]:
df

,english,portuguese,info,port_clean,eng_clean
1000,It's a TV.,É uma TV.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...,e uma tv,its a tv
1001,It's a TV.,Isso é uma TV.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...,isso e uma tv,its a tv
1002,It's a TV.,Isto é uma TV.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...,isto e uma tv,its a tv
1003,It's cool.,É legal.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...,e legal,its cool
1004,It's done!,"Pronto, já está!",CC-BY 2.0 (France) Attribution: tatoeba.org #1...,pronto ja esta,its done
...,...,...,...,...,...
19996,Are you a teacher?,Você é professor?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...,voce e professor,are you a teacher
19997,Are you a teacher?,Tu és professor?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...,tu es professor,are you a teacher
19998,Are you all right?,Você está bem?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...,voce esta bem,are you all right
19999,Are you all right?,Vocês estão bem?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...,voces estao bem,are you all right


In [11]:
# Tokenize words
pt_text_tokenized, pt_text_tokenizer = tokenize(df['port_clean'])
eng_text_tokenized, eng_text_tokenizer = tokenize(df['eng_clean'])

In [12]:
# Print das palavras Distintas
print('Maior sentença Português: {}'.format(len(max(pt_text_tokenized, key = len))))
print('Maior sentença Inglês: {}'.format(len(max(eng_text_tokenized, key = len))))

# Checa o tamanho dos vocabulários
port_vocab = len(pt_text_tokenizer.word_index) + 1
english_vocab = len(eng_text_tokenizer.word_index) + 1

# Print do tamanho dos Vocabulários
print("Vocabulário em Português têm {} palavras distintas".format(port_vocab))
print("Vocabulário em Inglês têm {} palavras distintas".format(english_vocab))

Maior sentença Português: 8
Maior sentença Inglês: 5
Vocabulário em Português têm 5655 palavras distintas
Vocabulário em Inglês têm 3269 palavras distintas


In [13]:
# salva os maiores valores
max_port_len = int(len(max(pt_text_tokenized, key = len)))
max_english_len = int(len(max(eng_text_tokenized, key = len)))

# Cria o pad sequences para as línguas
pt_pad_sentence = pad_sequences(pt_text_tokenized, max_port_len, padding = "post")
eng_pad_sentence = pad_sequences(eng_text_tokenized, max_english_len, padding = "post")

In [14]:
max_port_len

8

In [24]:
# Ajusta o shape dos dados
pt_pad_sentence = pt_pad_sentence.reshape(*pt_pad_sentence.shape, 1)
eng_pad_sentence = eng_pad_sentence.reshape(*eng_pad_sentence.shape, 1)

In [15]:
# Montagem das Camadas da Rede
# Camada de Entrada
input_sequence = Input(shape = (max_port_len,))
# Camada de Embedding
embedding = Embedding(input_dim = port_vocab,
                      output_dim = 128,)(input_sequence)
# Camada Encoder
encoder = LSTM(64,
               return_sequences = False)(embedding)
# Hidden State
r_vec = RepeatVector(max_english_len)(encoder)
# Decoder
decoder = LSTM(64,
               return_sequences = True,
               dropout = 0.2)(r_vec)
# Camada de Saída
logits = TimeDistributed(Dense(english_vocab))(decoder)

# Define a rede como um modelo
enc_dec_model = Model(input_sequence,
                      Activation('softmax')(logits))

# Compila a Rede
enc_dec_model.compile(loss = sparse_categorical_crossentropy,
                      optimizer = 'adam',
                      metrics = ['accuracy'])

# Sumário das Camadas
enc_dec_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
embedding (Embedding)        (None, 8, 128)            723840    
_________________________________________________________________
lstm (LSTM)                  (None, 64)                49408     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 5, 64)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 64)             33024     
_________________________________________________________________
time_distributed (TimeDistri (None, 5, 3269)           212485    
_________________________________________________________________
activation (Activation)      (None, 5, 3269)           0     

In [16]:
# Fit da Rede
model_results = enc_dec_model.fit(pt_pad_sentence,
                                  eng_pad_sentence,
                                  batch_size = 30, # Tamanho dos pacotes para treino
                                  epochs = 50) # número de iterações

2022-09-09 17:22:04.090380: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/50
634/634 [==============================] - 7s 9ms/step - loss: 4.3092 - accuracy: 0.3838
Epoch 2/50
634/634 [==============================] - 6s 9ms/step - loss: 3.6928 - accuracy: 0.4182
Epoch 3/50
634/634 [==============================] - 6s 9ms/step - loss: 3.4654 - accuracy: 0.4305
Epoch 4/50
634/634 [==============================] - 6s 9ms/step - loss: 3.2201 - accuracy: 0.4628
Epoch 5/50
634/634 [==============================] - 6s 10ms/step - loss: 2.9617 - accuracy: 0.5049
Epoch 6/50
634/634 [==============================] - 6s 10ms/step - loss: 2.7432 - accuracy: 0.5271
Epoch 7/50
634/634 [==============================] - 6s 10ms/step - loss: 2.5722 - accuracy: 0.5464
Epoch 8/50
634/634 [==============================] - 6s 10ms/step - loss: 2.4297 - accuracy: 0.5627
Epoch 9/50
634/634 [==============================] - 6s 10ms/step - loss: 2.3088 - accuracy: 0.5763
Epoch 10/50
634/634 [==============================] - 6s 9ms/step - loss: 2.1948 - accuracy: 0

In [34]:
# Função para converter os resultados nas sentences
def logits_to_sentence(logits, tokenizer):
    # Identifica as palavras dentro do tokenizer
    index_to_words = {idx: word for word, idx in tokenizer.word_index.items()}
    index_to_words[0] = '<empty>'

    # Retorna as predições
    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

# Índice para o teste
index = 42

# Resultados do Modelo
print("A sentença em Inglês é         : {}".format(df['english'].iloc[index]))
print("A sentença em Português é      : {}".format(df['portuguese'].iloc[index]))
print('A sentença predita pelo modelo :')
print(logits_to_sentence(enc_dec_model.predict(pt_pad_sentence[index:index+1])[0], eng_text_tokenizer))

A sentença em Inglês é         : Keep cool.
A sentença em Português é      : Te acalma.
A sentença predita pelo modelo :
keep cool <empty> <empty> <empty>
